In [2]:
from matplotlib import pyplot as plt
import matplotlib.lines as mlines
import numpy as np
plt.rcParams["figure.figsize"] = (20,10)

In [81]:
class Poly:
    """"
        c_0 + c_1 x + c_2 x^2 + .. + c_deg x^deg
    """
    def __init__(self, *, deg=2, coeffs=None):
        if coeffs is not None:
            self.coeffs = np.array(coeffs)
        else:
            self.coeffs = np.ones(deg+1)

    def __call__(self, x):
        return sum(c_i * x**i for (i, c_i) in enumerate(self.coeffs))

    def grad(self, x):
        return np.array([x**i for (i, c_ip1) in enumerate(self.coeffs[:])]).T[0]

    def hessian(self, x):
        return np.diag(list(x**i for (i, c_ip1) in enumerate(self.coeffs[:])))

In [94]:
def regression(x, y, method, **config):
    if config == {}:
        config = {"lr0": 0.5, "d": 0.005, "epoch": 1000}
    f = lambda x_poly: y - x_poly(x.T[0])
    jacobi = lambda x_poly: np.array([- x_poly.grad(x[i]) for i in range(len(x))])
    bs = method(f, np.zeros(len(x)), **config, jacobi=jacobi)
    print('hm')
    print(f'came close by {f(Poly(coeffs=bs[-1]))}')
    return bs[-1]

def gauss_newton(f, x, *, lr, epoch, jacobi):
    points = np.zeros((epoch, len(x)))
    x_poly = Poly(coeffs=x)
    points[0] = x_poly.coeffs

    for i in range(epoch):
        j = jacobi(x_poly)
        g = np.matmul(j.T, f(x_poly).reshape(-1,1))
        h = np.matmul(j.T,j)
        p = np.matmul(np.linalg.inv(h), g).T[0]
        x_poly.coeffs -= lr * p
        points[i] = x_poly.coeffs
    return points

regression(np.array([[-2], [3], [5]]), np.array([5, 10, 26]), gauss_newton, lr=1, epoch=2)

hm
came close by [-2.47268872e-12 -2.40163445e-12 -7.94386779e-12 -4.60431693e-12]


array([1.00000000e+00, 4.71844785e-15, 1.00000000e+00])